# 14wk-1: 강화학습 (2) – 4x4 Grid World

최규빈  
2024-06-03

<a href="https://colab.research.google.com/github/guebin/DL2024/blob/main/posts/14wk-1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

In [ ]:
# {{<video https://youtu.be/playlist?list=PLQqh36zP38-zHvVuJ92xfdypwHwDFgg8k&si=iI4IhthblTsJTmIv >}}

# 2. Imports

In [2]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.5 MB/s eta 0:00:00


In [3]:
#!pip install gymnasium
#---#
import gymnasium as gym
#---#
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import IPython

# 3. Game2: 4x4 Grid World

`-` 문제설명: 4x4 그리드월드에서 상하좌우로 움직이는 에이전트가 목표점에
도달하도록 학습하는 방법

`-` GridWorld에서 사용되는 주요변수

1.  **`State`**: 각 격자 셀이 하나의 상태이며, 에이전트는 이러한 상태 중
    하나에 있을 수 있음.
2.  **`Action`**: 에이전트는 현재상태에서 다음상태로 이동하기 위해
    상,하,좌,우 중 하나의 행동을 취할 수 있음.
3.  **`Reward`**: 에이전트가 현재상태에서 특정 action을 하면 얻어지는
    보상.
4.  **`Terminated`**: 하나의 에피소드가 종료되었음을 나타내는 상태. -> 상태를 초기화.

# 4. 예비학습



## A. `gym.spaces`
- action_space를 설정하는 것과 같음



`-` 예시1

In [4]:
state_space = gym.spaces.Discrete(4) #discrete 한 4개의 space
state_space  #action_space = [0,1,2,3] 과 같은 기능.

Discrete(4)

In [5]:
[state_space.sample() for _ in range(5)]

[1, 1, 3, 0, 3]

In [6]:
0 in state_space #0이 state_space에 있나?

True

In [7]:
4 in state_space

False

`-` 예시2

In [8]:
state_space = gym.spaces.MultiDiscrete([4,4]) #2 dimension.
state_space

MultiDiscrete([4 4])

In [9]:
[state_space.sample() for _ in range(5)]

[array([0, 1]), array([0, 3]), array([0, 3]), array([0, 0]), array([1, 3])]

In [10]:
np.array([3,3]) in state_space #numpy array가 호환된다.

True

In [11]:
np.array([3,4]) in state_space #[0,0] ~ [3,3]

False

## B. 시각화

In [12]:
def show(states): #환경의 동작을 구현..
    fig = plt.Figure()
    ax = fig.subplots()
    ax.matshow(np.zeros([4,4]), cmap='bwr',alpha=0.0)
    sc = ax.scatter(0, 0, color='red', s=500)
    ax.text(0, 0, 'start', ha='center', va='center')
    ax.text(3, 3, 'end', ha='center', va='center')
    # Adding grid lines to the plot
    ax.set_xticks(np.arange(-.5, 4, 1), minor=True)
    ax.set_yticks(np.arange(-.5, 4, 1), minor=True)
    ax.grid(which='minor', color='black', linestyle='-', linewidth=2)
    state_space = gym.spaces.MultiDiscrete([4,4])
    def update(t):
        if states[t] in state_space:
            s1,s2 = states[t]
            states[t] = [s2,s1]
            sc.set_offsets(states[t])
        else:
            s1,s2 = states[t]
            s1 = s1 + 0.5 if s1 < 0 else (s1 - 0.5 if s1 > 3 else s1)
            s2 = s2 + 0.5 if s2 < 0 else (s2 - 0.5 if s2 > 3 else s2)
            states[t] = [s2,s1]
            sc.set_offsets(states[t])
    ani = FuncAnimation(fig,update,frames=len(states))
    display(IPython.display.HTML(ani.to_jshtml()))

In [13]:
show([[0,0],
      [1,0],
      [2,0],
      [3,0],
      [4,0]])

# 5. Env 클래스 구현

(현재 상태, action) -step-> (다음 상태,보상)\
Agent는 action을 env에게 주고, env는 다음 상태와 보상을 준다.

In [14]:
action = 3
current_state = np.array([1,1]) #초기 위치 지정
#수업에서는 state_space.sample()

In [38]:
action_to_direction = {
    0 : np.array([1, 0]), # row+, down
    1 : np.array([0, 1]), # col+, right
    2 : np.array([-1 ,0]), # row-, up
    3 : np.array([0, -1]) # col-, left
}
action_to_direction2 = {0: 'down', 1: 'right', 2: 'up', 3: 'left'} #자연어 버전

현재상태 + 헹동의 numpy version -> 다음상태

In [16]:
action_to_direction[action] #강의에선 얘를 direction으로 저장.

array([ 0, -1])

In [17]:
next_state = current_state + action_to_direction[action]
next_state #강의에선 current_state + direction으로 저장.

array([1, 0])

`-` Class 구현: 아래와 같은 느낌의 클래스를 구현해보자.

In [39]:
class GridWorld:
    def __init__(self): #environment를 초기화 할 떄 받아야할 정보들.
        self.reset() #실행하면 state가 나옴.
        self.state_space = gym.spaces.MultiDiscrete([4,4])
        self.action_space = gym.spaces.Discrete(4) #actionspace도 environment에 지정하는 경우도 있다.
        self._action_to_direction = {
            0 : np.array([1, 0]), # x+
            1 : np.array([0, 1]), # y+
            2 : np.array([-1 ,0]), # x-
            3 : np.array([0, -1]) # y-
        } #init은 생각보다 늦은 순서에 설정... step의 return 고려 후 init 고려
    def step(self,action):
        direction = self._action_to_direction[action]
        self.state = self.state + direction #이건 next_state와 같음
        if self.state not in self.state_space: # 4x4 그리드 밖에 있는 경우
            reward = -10
            terminated = True
        elif np.array_equal(self.state, np.array([3,3])): # 목표지점에 도달할 경우
         #(np.array_equal이라는 메소드가 있다..) 두 array가 같냐 다르냐에 대해 bool 자료를 반환
            reward = 100
            terminated = True
        else:
            reward = -1 #빨리 clear하게끔 압박하는 수단.
            terminated = False
        return self.state, reward, terminated #무엇을 반환해야할지 먼저 생각하면 코드짤 떄 좋은 것 같음
    def reset(self): #
        self.terminated = False
        self.state = np.array([0,0])
        return self.state

#코드를 짜는 흐름이 step부터 정의를 하고 __init__에 저장이 필요한 자료들을 정리함.

In [40]:
env = GridWorld()

In [41]:
states = []
state = env.reset()

In [42]:
state #리셋한 결과로 [0,0]이 나오는 것임.

array([0, 0])

In [22]:
states.append(state)  #원점을 states에 추가

In [23]:
states

[array([0, 0])]

In [43]:
for _ in range(50):
    action = env.action_space.sample()
    state,reward,terminated = env.step(action) #return값 언패킹
    states.append(state) #state 갱신.
    if terminated: break #True 일 때 Break

In [45]:
states

[array([0, 1]), array([0, 2]), array([0, 3]), array([-1,  3])]

In [26]:
show(states)

# 6. `AgentRandom`



## A. 에이전트 클래스 설계

`-` 우리가 구현하고 싶은 기능

-   `.act()`: 액션을 결정 –\> 여기서는 그냥 랜덤액션
-   `.save_experience()`: 데이터를 저장 –\> 여기에 일단 초점을 맞추자
-   `.learn()`: 데이터로에서 학습 –\> 패스



`-` 첫번째 시도

In [46]:
class AgentRandom: #잡다하다..
    def __init__(self,env): #2. init에 env 입력.
        # define spaces
        self.action_space = env.action_space #1. 난 env가 뭔지 몰라
        self.state_space = env.state_space
        # other info
        #self.score = 0
        #self.scores = [] #에피소드별 점수 기록.
        #self.playtimes = [] #게임별 플레이타임 기록.
        self.n_episodes = 0
        self.n_experiences = 0
        # replay_buffer
        self.current_state = None
        self.action = None
        self.reward = None
        self.next_state = None
        self.terminated = None
        #--#
        self.current_states = []
        self.actions = []
        self.rewards = []
        self.next_states = []
        self.terminations = []
    def act(self):
        self.action = self.action_space.sample() #지금은 랜덤.
    def save_experience(self):
        self.actions.append(self.action)
        self.current_states.append(self.current_state)
        self.rewards.append(self.reward)
        self.next_states.append(self.next_state)
        self.terminations.append(self.terminated)
        #--#
        self.n_experiences += 1 #n_ex = n_ex + 1 과 같은 코드
        #self.score = self.score + self.reward

    def learn(self):
        pass

#### 작동방식 구경

In [28]:
env = GridWorld()

In [53]:
env.reset() #리셋 하고 ( 밑에서 죽으면 reset 해줘야 함 )

array([0, 0])

In [54]:
agent = AgentRandom(env) #에이전트 객체 만들고

In [55]:
agent.act() #한번 action
env.step(agent.action) #action에 따른 결과 (다음 state, reward, terminate 여부)

In [56]:
agent.action #어떤 action인지 확인

1

In [57]:
env.step(agent.action) #action에 따른 결과 (다음 state, reward, terminate 여부)

(array([0, 1]), -1, False)

In [58]:
#안죽었다면?
agent.act() #한번 action
env.step(agent.action) #action에 따른 결과 (다음 state, reward, terminate 여부)

(array([0, 0]), -1, False)

In [63]:
env = GridWorld()
agent = AgentRandom(env)
states = []
states.append(env.reset())

#한번의 에피소드
for t in range(1000):
  # step1
  agent.act()
  # step2
  agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
  # step3
  agent.save_experience()
  #agent.learn() #아직 구현 x
  if agent.terminated: #terminated == True 라면
    env.reset()
    break

In [61]:
agent.current_states

[array([1, 0]), array([2, 0]), array([ 2, -1])]

끊김.. 이어서

In [67]:
env = GridWorld()
agent = AgentRandom(env)

#10번의 에피소드
for _ in range(10):
  agent.current_state = env.reset()
  for t in range(1000):
    # step1 행동
    agent.act()
    # step2 보상
    agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
    # step3 저장& 학습 (학습은 pass)
    agent.save_experience()
    agent.learn() #아직 구현 x
    # step4
    agent.current_state = agent.next_state
    if agent.terminated: #terminated == True 라면
      break

In [69]:
agent.current_states[:5] #일단 5번만 보자 #4. 갱신된다.
#종료시점이 next_states에 포함되는 경향이 있고
#시작시점이 current_states에 포함되는 경향이 있다.

[array([0, 0]), array([0, 1]), array([0, 0]), array([0, 1]), array([0, 0])]

In [70]:
agent.actions[:5] #1. 이러한 액션들의 결과

[1, 3, 1, 2, 3]

In [71]:
env._action_to_direction #2. 이러한 방향으로 움직이고

{0: array([1, 0]), 1: array([0, 1]), 2: array([-1,  0]), 3: array([ 0, -1])}

In [72]:
agent.next_states[:5] #3. 이러한 위치에 도달하면

[array([0, 1]), array([0, 0]), array([0, 1]), array([-1,  1]), array([ 0, -1])]

## B. 환경과 상호작용

이건 위에서 구현한 내용을 반복수에 따라 state를 보여주는 코드

#### 정리할 때 예전 코드로 넘길것

In [34]:
env = GridWorld()
agent = AgentRandom(env)
for _ in range(20):
    # Step1: 에피소드 준비
    agent.current_state = env.reset()
    agent.terminated = False
    agent.score = 0
    # Step2: 에프소드 진행
    for t in range(50):
        # step1: 행동
        agent.act()
        # step2: 보상
        agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
        # step3: 저장 & 학습
        agent.save_experience()
        # agent.learn()
        # step4: 다음 스텝준비
        agent.current_state = agent.next_state
        if agent.terminated: break
    # Step3: 다음에피소드 준비
    agent.scores.append(agent.score)
    agent.playtimes.append(t+1)
    agent.n_episodes = agent.n_episodes + 1
    #--#
    print(
        f"Epsiode: {agent.n_episodes} \t"
        f"Score: {agent.scores[-1]} \t"
        f"Playtime: {agent.playtimes[-1]}"
    )

Epsiode: 1 	Score: -14 	Playtime: 5
Epsiode: 2 	Score: -10 	Playtime: 1
Epsiode: 3 	Score: -10 	Playtime: 1
Epsiode: 4 	Score: -10 	Playtime: 1
Epsiode: 5 	Score: -10 	Playtime: 1
Epsiode: 6 	Score: -10 	Playtime: 1
Epsiode: 7 	Score: -10 	Playtime: 1
Epsiode: 8 	Score: -10 	Playtime: 1
Epsiode: 9 	Score: -12 	Playtime: 3
Epsiode: 10 	Score: -13 	Playtime: 4
Epsiode: 11 	Score: -19 	Playtime: 10
Epsiode: 12 	Score: -12 	Playtime: 3
Epsiode: 13 	Score: -10 	Playtime: 1
Epsiode: 14 	Score: -21 	Playtime: 12
Epsiode: 15 	Score: -10 	Playtime: 1
Epsiode: 16 	Score: -11 	Playtime: 2
Epsiode: 17 	Score: -12 	Playtime: 3
Epsiode: 18 	Score: -17 	Playtime: 8
Epsiode: 19 	Score: -10 	Playtime: 1
Epsiode: 20 	Score: -18 	Playtime: 9


## C. 상호작용결과 시각화

**현재 강의 노트**

In [73]:
agent.next_states #그때 그떄 관찰해야하긴 하는데...

[array([0, 1]),
 array([0, 0]),
 array([0, 1]),
 array([-1,  1]),
 array([ 0, -1]),
 array([ 0, -1]),
 array([0, 1]),
 array([1, 1]),
 array([1, 2]),
 array([1, 3]),
 array([2, 3]),
 array([2, 2]),
 array([1, 2]),
 array([2, 2]),
 array([3, 2]),
 array([3, 3]),
 array([-1,  0]),
 array([1, 0]),
 array([1, 1]),
 array([1, 2]),
 array([1, 1]),
 array([1, 0]),
 array([1, 1]),
 array([1, 0]),
 array([0, 0]),
 array([1, 0]),
 array([2, 0]),
 array([ 2, -1]),
 array([-1,  0]),
 array([-1,  0]),
 array([0, 1]),
 array([0, 0]),
 array([ 0, -1]),
 array([1, 0]),
 array([1, 1]),
 array([0, 1]),
 array([0, 0]),
 array([-1,  0])]

In [88]:
[np.array([0,0])] + agent.next_states[:4] #한 에피소드에 대해 첫시점 , 종료까지

[array([0, 0]), array([0, 1]), array([0, 0]), array([0, 1]), array([-1,  1])]

In [89]:
[np.array([0,0])] + [agent.next_states[4]] #두번째 에피소드에 대해 첫시점 , 종료까지

[array([0, 0]), array([ 0, -1])]

In [90]:
[np.array([0,0])] + [agent.next_states[5]] #세번째 에피소드에 대해 첫시점 , 종료까지

[array([0, 0]), array([ 0, -1])]

In [91]:
[np.array([0,0])] + agent.next_states[6:16] #네번째 에피소드에 대해 첫시점 , 종료까지

[array([0, 0]),
 array([0, 1]),
 array([1, 1]),
 array([1, 2]),
 array([1, 3]),
 array([2, 3]),
 array([2, 2]),
 array([1, 2]),
 array([2, 2]),
 array([3, 2]),
 array([3, 3])]

In [92]:
[np.array([0,0])] + [agent.next_states[16]] #다섯번째 에피소드에 대해 첫시점 , 종료까지

[array([0, 0]), array([-1,  0])]

In [94]:
show([np.array([0,0])] + agent.next_states[6:16])

이딴식으로 일일이 잘라서 한다면 너무 소모적...

대안: playtimes와 scores를 적절히 활용하자!

In [96]:
class AgentRandom: #잡다하다..
    def __init__(self,env): #2. init에 env 입력.
        # define spaces
        self.action_space = env.action_space #1. 난 env가 뭔지 몰라
        self.state_space = env.state_space
        # other info
        self.score = 0
        self.scores = [] #에피소드별 점수 기록.
        self.playtimes = [] #게임별 플레이타임 기록.
        self.n_episodes = 0
        self.n_experiences = 0
        # replay_buffer (sarsa 에서 sars 가 들어감)
        self.current_state = None
        self.action = None
        self.reward = None
        self.next_state = None
        self.terminated = None
        #--# 핵심.
        self.current_states = []
        self.actions = []
        self.rewards = []
        self.next_states = []
        self.terminations = []
    def act(self):
        self.action = self.action_space.sample() #지금은 랜덤.
    def save_experience(self):
        self.actions.append(self.action)
        self.current_states.append(self.current_state)
        self.rewards.append(self.reward)
        self.next_states.append(self.next_state)
        self.terminations.append(self.terminated)
        #위는 환경이랑 상호작용하며 얻는 것들.
        #--#
        #아래는 agent의 편의상 기록하는 것들.
        self.n_experiences += 1 #n_ex = n_ex + 1 과 같은 코드
        self.score = self.score + self.reward

    def learn(self):
        pass

작동 방식을 다음처럼 보자\
`sarsa` algorithm..:(s,a) (r,s') a' ; s,a,r는 약어

In [108]:
env = GridWorld()
agent = AgentRandom(env)

#10번의 에피소드
for _ in range(1): #일단 episode 1만 관찰
  agent.current_state = env.reset()
  for t in range(1000):
    # step1 행동
    agent.act()
    # step2 보상
    agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
    # step3 저장& 학습 (학습은 pass)
    agent.save_experience()
    agent.learn() #아직 구현 x
    # step4
    agent.current_state = agent.next_state
    if agent.terminated: #terminated == True 라면
      break
  print(agent.score) #agent.score는 한판 할 떄 reward

-10


In [117]:
env = GridWorld()
agent = AgentRandom(env)

#10번의 에피소드
for _ in range(50): #50번의 에피소드 관
  agent.current_state = env.reset()
  agent.score = 0
  for t in range(1000):
    # step1 행동
    agent.act()
    # step2 보상
    agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
    # step3 저장& 학습 (학습은 pass)
    agent.save_experience()
    agent.learn() #아직 구현 x
    # step4
    agent.current_state = agent.next_state
    if agent.terminated: #terminated == True 라면
      break
  agent.scores.append(agent.score) #agent.score는 한판 할 떄 reward
  agent.playtimes.append(t+1)

In [120]:
#잡기술 게임 플레이 횟수, 보상점수, 시행 횟수
{i:[s,p] for i,(s,p) in enumerate(zip(agent.scores, agent.playtimes))}
#간간히 클리어 하는 경우가 보임

{0: [-10, 1],
 1: [-11, 2],
 2: [-12, 3],
 3: [-10, 1],
 4: [-13, 4],
 5: [-11, 2],
 6: [-10, 1],
 7: [-19, 10],
 8: [-15, 6],
 9: [-10, 1],
 10: [-11, 2],
 11: [-10, 1],
 12: [-13, 4],
 13: [-10, 1],
 14: [-14, 5],
 15: [-12, 3],
 16: [-10, 1],
 17: [-11, 2],
 18: [-10, 1],
 19: [-14, 5],
 20: [-10, 1],
 21: [-19, 10],
 22: [-33, 24],
 23: [-10, 1],
 24: [-12, 3],
 25: [-14, 5],
 26: [-12, 3],
 27: [-13, 4],
 28: [-11, 2],
 29: [-26, 17],
 30: [-14, 5],
 31: [-11, 2],
 32: [-10, 1],
 33: [-10, 1],
 34: [-15, 6],
 35: [-10, 1],
 36: [-10, 1],
 37: [-10, 1],
 38: [-10, 1],
 39: [-15, 6],
 40: [-14, 5],
 41: [-15, 6],
 42: [-11, 2],
 43: [-12, 3],
 44: [-12, 3],
 45: [-10, 1],
 46: [-15, 6],
 47: [-10, 1],
 48: [-10, 1],
 49: [-10, 1]}

#### 최종!

In [129]:
env = GridWorld()
agent = AgentRandom(env)

#10번의 에피소드
for _ in range(50): #50번의 에피소드 관
  agent.current_state = env.reset()
  agent.score = 0
  for t in range(1000):
    # step1 행동
    agent.act()
    # step2 보상
    agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
    # step3 저장& 학습 (학습은 pass)
    agent.save_experience()
    agent.learn() #아직 구현 x
    # step4
    agent.current_state = agent.next_state
    if agent.terminated: #terminated == True 라면
      break
  agent.scores.append(agent.score) #agent.score는 한판 할 떄 reward
  agent.playtimes.append(t+1)
  #---추가
  agent.n_episodes = agent.n_episodes + 1
  print(
        f"Epsiode: {agent.n_episodes} \t"
        f"Score: {agent.scores[-1]} \t"
        f"Playtime: {agent.playtimes[-1]}\t"
        f"경험: {agent.n_experiences}"
        )

Epsiode: 1 	Score: -10 	Playtime: 1	경험: 1
Epsiode: 2 	Score: -10 	Playtime: 1	경험: 2
Epsiode: 3 	Score: -10 	Playtime: 1	경험: 3
Epsiode: 4 	Score: -15 	Playtime: 6	경험: 9
Epsiode: 5 	Score: 95 	Playtime: 6	경험: 15
Epsiode: 6 	Score: 93 	Playtime: 8	경험: 23
Epsiode: 7 	Score: -10 	Playtime: 1	경험: 24
Epsiode: 8 	Score: -10 	Playtime: 1	경험: 25
Epsiode: 9 	Score: -10 	Playtime: 1	경험: 26
Epsiode: 10 	Score: -21 	Playtime: 12	경험: 38
Epsiode: 11 	Score: -10 	Playtime: 1	경험: 39
Epsiode: 12 	Score: -11 	Playtime: 2	경험: 41
Epsiode: 13 	Score: -11 	Playtime: 2	경험: 43
Epsiode: 14 	Score: -10 	Playtime: 1	경험: 44
Epsiode: 15 	Score: -10 	Playtime: 1	경험: 45
Epsiode: 16 	Score: -27 	Playtime: 18	경험: 63
Epsiode: 17 	Score: -14 	Playtime: 5	경험: 68
Epsiode: 18 	Score: -12 	Playtime: 3	경험: 71
Epsiode: 19 	Score: -10 	Playtime: 1	경험: 72
Epsiode: 20 	Score: -16 	Playtime: 7	경험: 79
Epsiode: 21 	Score: -14 	Playtime: 5	경험: 84
Epsiode: 22 	Score: -13 	Playtime: 4	경험: 88
Epsiode: 23 	Score: -15 	Playtime: 6	경험: 94
E

##### 예전 강의 노트

In [35]:
sum(agent.playtimes[:18])

59

In [36]:
sum(agent.playtimes[:19])

60

In [37]:
states = [np.array([0,0])] + agent.next_states[76:84]
show(states)

-   우연히 잘맞춘 케이스



# 7. `AgentGreedy` 다음 시간부터

## A. 환경의 이해



`-` 무작위로 10000판을 진행해보자.

In [ ]:
for _ in range(10000):
    # Step1: 에피소드 준비
    agent.current_state = env.reset()
    agent.terminated = False
    agent.score = 0
    # Step2: 에프소드 진행
    for t in range(50):
        # step1: 행동
        agent.act()
        # step2: 보상
        agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
        # step3: 저장 & 학습
        agent.save_experience()
        # agent.learn()
        # step4: 다음 스텝준비
        agent.current_state = agent.next_state
        if agent.terminated: break
    # Step3: 다음에피소드 준비
    agent.scores.append(agent.score)
    agent.playtimes.append(t+1)
    agent.n_episodes = agent.n_episodes + 1

In [ ]:
agent.n_experiences

32902

`-` 데이터관찰

In [ ]:
print(f"에이전트: 현재상태/행동 = {agent.current_states[0]} / {agent.actions[0]}")
print(f"환경: 보상/다음상태 = {agent.rewards[0]} / {agent.next_states[0]}")

에이전트: 현재상태/행동 = [0 0] / 2
환경: 보상/다음상태 = -10 / [-1  0]


In [ ]:
print(f"에이전트: 현재상태/행동 = {agent.current_states[1]} / {agent.actions[1]}")
print(f"환경: 보상/다음상태 = {agent.rewards[1]} / {agent.next_states[1]}")

에이전트: 현재상태/행동 = [0 0] / 2
환경: 보상/다음상태 = -10 / [-1  0]


In [ ]:
print(f"에이전트: 현재상태/행동 = {agent.current_states[2]} / {agent.actions[2]}")
print(f"환경: 보상/다음상태 = {agent.rewards[2]} / {agent.next_states[2]}")

에이전트: 현재상태/행동 = [0 0] / 2
환경: 보상/다음상태 = -10 / [-1  0]


In [ ]:
print(f"에이전트: 현재상태/행동 = {agent.current_states[3]} / {agent.actions[3]}")
print(f"환경: 보상/다음상태 = {agent.rewards[3]} / {agent.next_states[3]}")

에이전트: 현재상태/행동 = [0 0] / 1
환경: 보상/다음상태 = -1 / [0 1]


In [ ]:
print(f"에이전트: 현재상태/행동 = {agent.current_states[4]} / {agent.actions[4]}")
print(f"환경: 보상/다음상태 = {agent.rewards[4]} / {agent.next_states[4]}")

에이전트: 현재상태/행동 = [0 1] / 2
환경: 보상/다음상태 = -10 / [-1  1]


`-` 환경을 이해하기 위한 기록 (1)

In [ ]:
q_table = np.zeros([4,4,4])
count = np.zeros([4,4,4])
for i in range(agent.n_experiences):
    s1,s2 = agent.current_states[i]
    a = agent.actions[i]
    q_table[s1,s2,a] = q_table[s1,s2,a] + agent.rewards[i]
    count[s1,s2,a] = count[s1,s2,a] + 1

In [ ]:
count[count == 0] = 0.01
q_table = q_table/count

In [ ]:
q_table[:,:,3]

array([[-10.,  -1.,  -1.,  -1.],
       [-10.,  -1.,  -1.,  -1.],
       [-10.,  -1.,  -1.,  -1.],
       [-10.,  -1.,  -1.,   0.]])

In [ ]:
for a in range(4):
    print(
        f"action = {a}/{action_to_direction2[a]}\n"
        f"action-value function = \n {q_table[:,:,a].round(3)}\n"
)

action = 0/down
action-value function = 
 [[ -1.  -1.  -1.  -1.]
 [ -1.  -1.  -1.  -1.]
 [ -1.  -1.  -1. 100.]
 [-10. -10. -10.   0.]]

action = 1/right
action-value function = 
 [[ -1.  -1.  -1. -10.]
 [ -1.  -1.  -1. -10.]
 [ -1.  -1.  -1. -10.]
 [ -1.  -1. 100.   0.]]

action = 2/up
action-value function = 
 [[-10. -10. -10. -10.]
 [ -1.  -1.  -1.  -1.]
 [ -1.  -1.  -1.  -1.]
 [ -1.  -1.  -1.   0.]]

action = 3/left
action-value function = 
 [[-10.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.]
 [-10.  -1.  -1.   0.]]



`-` 환경을 이해하기 위한 기록 (2)

In [ ]:
q_table = np.zeros([4,4,4])
for i in range(agent.n_experiences):
    s1,s2 = agent.current_states[i]
    a = agent.actions[i]
    q_hat = q_table[s1,s2,a] # 우리가 환경을 이해하고 있는 값, 우리가 풀어낸 답
    q = agent.rewards[i] # 실제 답
    diff = q - q_hat # 실제답과 풀이한값의 차이 = 오차피드백값
    q_table[s1,s2,a] = q_hat + 0.05 * diff ## 새로운답 = 원래답 + 오차피드백값

In [ ]:
q_table[:,:,0]

array([[-1.        , -1.        , -1.        , -0.99891047],
       [-1.        , -1.        , -1.        , -0.99962895],
       [-0.99999999, -0.99999999, -0.99999208, 99.27311433],
       [-9.98662342, -9.99156942, -9.95647867,  0.        ]])

In [ ]:
for a in range(4):
    print(
        f"action = {a}/{action_to_direction2[a]}\n"
        f"action-value function = \n {q_table[:,:,a].round(2)}\n"
)

action = 0/down
action-value function = 
 [[-1.   -1.   -1.   -1.  ]
 [-1.   -1.   -1.   -1.  ]
 [-1.   -1.   -1.   99.27]
 [-9.99 -9.99 -9.96  0.  ]]

action = 1/right
action-value function = 
 [[ -1.    -1.    -1.   -10.  ]
 [ -1.    -1.    -1.   -10.  ]
 [ -1.    -1.    -1.    -9.93]
 [ -1.    -1.    97.87   0.  ]]

action = 2/up
action-value function = 
 [[-10.   -10.   -10.    -9.97]
 [ -1.    -1.    -1.    -1.  ]
 [ -1.    -1.    -1.    -1.  ]
 [ -1.    -1.    -0.99   0.  ]]

action = 3/left
action-value function = 
 [[-10.    -1.    -1.    -1.  ]
 [-10.    -1.    -1.    -1.  ]
 [-10.    -1.    -1.    -0.99]
 [ -9.98  -1.    -1.     0.  ]]



## B. 환경의 깊은 이해

`-` action=1 일때 각 state의 가치 (=기대보상)

In [ ]:
q_table[:,:,1]

array([[-1.        , -1.        , -1.        , -9.99544445],
       [-1.        , -1.        , -1.        , -9.99753836],
       [-0.99999999, -0.99999999, -0.99999614, -9.92731143],
       [-0.99843982, -0.99956722, 97.86562662,  0.        ]])

`-` 분석1

In [ ]:
q_table[3,2,1]

97.86562661541224

-   상태 (3,2)에서 행동 1을 하게되면 100의 보상을 얻으므로 기대보상값은
    100근처 –\> 합리적임

`-` 분석2

In [ ]:
q_table[3,1,1]

-0.9995672227742406

-   상태 (3,1)에서 행동 1을 하게되면 -1 의 보상을 얻으므로 기대보상값은
    -1 근처 –\> 합리적일까??

`-` 비판: 분석2는 합리적인것 처럼 보이지만 data를 분석한 뒤에는 그다지
합리적이지 못함.

`-` 상황상상

-   빈 종이를 줌
-   빈 종이에는 0 또는 1을 쓸 수 있음 (action = 0 혹은 1)
-   0을 쓸때와 1을 쓸때 보상이 다름
-   무수히 많은 데이터를 분석해보니, 0을 쓰면 0원을 주고 1을 쓰면
    10만원을 보상을 준다는 것을 “알게 되었음”
-   이때 빈 종이의 가치는 5만원인가? 10만원인가? –\> 10만원아니야?

`-` 직관: 생각해보니 현재 $s=(3,1)$ $a=1$에서 추정된(esitated) 값은
`q_table[3,1,1]` $\approx$ -1 이지만[1], 현실적으로는 “실제보상(-1)과
잠재적보상(100)”을 동시에 고려해야 하는게 합리적임

[1] 즉 next_state가 가지는 잠재적값어치는 고려되어있지 않음

In [ ]:
q_hat = q_table[3,1,1]
q_hat

-0.9995672227742406

In [ ]:
q = (-1) + 0.99 * 100
q

98.0

-   여기에서 0.99는 “미래에 받을 보상이 현재에 비해 얼마나 중요한지를
    결정하는 가중치” 이다.
-   1에 가까울수록 미래에 받을 보상을 매우 중시한다는 의미 (즉 빈종이
    $\approx$ 십만원 으로 생각한다는 의미)
-   0.99는 보통 $\gamma$라는 기호로 표기하며 `discount rate`이라고
    표현한다. (외우세여)

`-` 즉 $q(s,a)$는 모든 $s$, $a$에 대하여

$$q(s,a) \approx \text{reward}(s,a) + 0.99 \times \max_{a}q(s',a)$$

가 성립한다면 $q(s,a)$는 타당하게 추정된 것이라 볼 수 있다. 물론 수식을
좀 더 엄밀하게 쓰면 (terminated, not-terminated 로 나누어 쓰면) 아래와
같다.

$$q(s,a) \approx \begin{cases}  \text{reward}(s,a) + 0.99 \times \max_{a}q(s',a) & \text{not terminated} \\ \text{reward}(s,a) & \text{terminated} \end{cases}$$

> **Note**
>
> 대충 설명하면서 넘어갔지만 이 수식을 **벨만방정식**이라고 부른다.
> (외우세여) 위의 식은 강화학습에서 가장 중요한 식이며 원래 버전은
> 아래와 같다.
>
> $$Q^\star(s,a) = R(s,a) +\gamma\sum_{s'}P(s'|s,a)\max_{a}Q(s',a)$$
>
> 여기에서 $P(s'|s,a)$ 는 상태 $s \in {\cal S}$에서 행동
> $a \in {\cal A}$를 했을때 $s'$에 있을 확률이다. 이러한 확률은
> “바람,소용돌이” 등의 외부의 확률적인 요소가 있는 환경에서 의미가
> 있으며 우리의 예제에서는 의미가 없다.

In [ ]:
q_table = np.zeros([4,4,4])
for i in range(agent.n_experiences):
    s1,s2 = agent.current_states[i]
    ss1,ss2 = agent.next_states[i]
    a = agent.actions[i]
    q_hat = q_table[s1,s2,a]
    if agent.terminations[i]:
        q = agent.rewards[i]
    else:
        future_reward = q_table[ss1,ss2,:].max()
        q = agent.rewards[i] + 0.99 * future_reward
    diff = q - q_hat
    q_table[s1,s2,a] = q_hat + 0.05 * diff

In [ ]:
for a in range(4):
    print(
        f"action = {a}/{action_to_direction2[a]}\n"
        f"action-value function = \n {q_table[:,:,a].round(2)}\n"
)

action = 0/down
action-value function = 
 [[88.19 90.18 92.05 91.43]
 [89.72 92.15 94.42 96.13]
 [81.95 90.72 94.44 99.27]
 [-9.99 -9.99 -9.96  0.  ]]

action = 1/right
action-value function = 
 [[ 88.22  89.95  87.62 -10.  ]
 [ 90.15  92.25  93.7  -10.  ]
 [ 91.98  94.34  96.8   -9.93]
 [ 84.82  93.72  97.87   0.  ]]

action = 2/up
action-value function = 
 [[-10.   -10.   -10.    -9.97]
 [ 86.07  88.    89.26  82.86]
 [ 87.57  89.74  91.6   88.48]
 [ 86.14  89.79  88.4    0.  ]]

action = 3/left
action-value function = 
 [[-10.    86.19  87.65  86.97]
 [-10.    87.96  89.75  89.94]
 [-10.    89.26  90.91  90.82]
 [ -9.98  76.92  84.05   0.  ]]



## C. 행동 전략 수립

`-` 상태 (0,0)에 있다고 가정해보자.

In [ ]:
q_table[0,0,:]

array([ 88.1877238,  88.2172839, -10.       , -10.       ])

-   행동 0 혹은 행동 1을 하는게 유리하다. // 행동 2,3을 하면 망한다.

`-` 상태 (2,3)에 있다고 가정해보자.

In [ ]:
q_table[2,3,:]

array([99.27311433, -9.92731143, 88.47723328, 90.82427941])

-   행동 0을 하는게 유리함.

`-` 상태 (3,2)에 있다고 가정해보자.

In [ ]:
q_table[3,2,:]

array([-9.95647867, 97.86562662, 88.4037384 , 84.05418085])

-   행동1을 하는게 유리함

`-` 각 상태에서 최적은 action은 아래와 같다.

In [ ]:
q_table[0,0,:].argmax()

1

In [ ]:
q_table[2,3,:].argmax()

0

In [ ]:
q_table[3,2,:].argmax()

1

`-` 전략(=정책)을 정리해보자.

In [ ]:
policy = np.array(['?????']*16).reshape(4,4)
policy

array([['?????', '?????', '?????', '?????'],
       ['?????', '?????', '?????', '?????'],
       ['?????', '?????', '?????', '?????'],
       ['?????', '?????', '?????', '?????']], dtype='<U5')

In [ ]:
for s1 in range(4):
    for s2 in range(4):
        policy[s1,s2] = action_to_direction2[q_table[s1,s2,:].argmax()]
policy

array([['right', 'down', 'down', 'down'],
       ['right', 'right', 'down', 'down'],
       ['right', 'right', 'right', 'down'],
       ['up', 'right', 'right', 'down']], dtype='<U5')

In [ ]:
q_table.max(axis=-1)

array([[88.2172839 , 90.18097353, 92.05187923, 91.4335475 ],
       [90.14546519, 92.24698809, 94.42022192, 96.13203251],
       [91.97788374, 94.33692116, 96.80198987, 99.27311433],
       [86.14407457, 93.72419685, 97.86562662,  0.        ]])

## D. 에이전트 클래스 설계

In [ ]:
class AgentGreedy(AgentRandom):
    def __init__(self,env):
        super().__init__(env)
        self.q_table = np.zeros([4,4,4])
    def learn(self):
        s1,s2 = self.current_state
        ss2,ss2 = self.next_state
        a = self.action
        q_hat = self.q_table[s1,s2,a]
        if self.terminated:
            q = self.reward
        else:
            future_reward = q_table[ss1,ss2,:].max()
            q = self.reward + 0.99 * future_reward
        diff = q - q_hat
        self.q_table[s1,s2,a] = q_hat + 0.05 * diff
    def act(self):
        if self.n_experiences < 3000:
            self.action = self.action_space.sample()
        else:
            s1,s2 = self.current_state
            self.action = self.q_table[s1,s2,:].argmax()

## E. 환경과 상호작용

In [ ]:
env = GridWorld()
agent = AgentGreedy(env)
for _ in range(5000):
    # Step1: 에피소드 준비
    agent.current_state = env.reset()
    agent.terminated = False
    agent.score = 0
    # Step2: 에프소드 진행
    for t in range(50):
        # step1: 행동
        agent.act()
        # step2: 보상
        agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
        # step3: 저장 & 학습
        agent.save_experience()
        agent.learn()
        # step4: 다음 스텝준비
        agent.current_state = agent.next_state
        if agent.terminated: break
    # Step3: 다음에피소드 준비
    agent.scores.append(agent.score)
    agent.playtimes.append(t+1)
    agent.n_episodes = agent.n_episodes + 1
    if (agent.n_episodes % 500) ==0:
        print(
            f"Epsiode: {agent.n_episodes} \t"
            f"Score: {np.mean(agent.scores[-100:])} \t"
            f"Playtime: {np.mean(agent.playtimes[-100:])}\t"
        )

Epsiode: 500 	Score: -10.33 	Playtime: 3.53	
Epsiode: 1000 	Score: -32.99 	Playtime: 31.68	
Epsiode: 1500 	Score: -50.0 	Playtime: 50.0	
Epsiode: 2000 	Score: -50.0 	Playtime: 50.0	
Epsiode: 2500 	Score: -50.0 	Playtime: 50.0	
Epsiode: 3000 	Score: -50.0 	Playtime: 50.0	
Epsiode: 3500 	Score: -50.0 	Playtime: 50.0	
Epsiode: 4000 	Score: -50.0 	Playtime: 50.0	
Epsiode: 4500 	Score: -50.0 	Playtime: 50.0	
Epsiode: 5000 	Score: -50.0 	Playtime: 50.0	


## F. 상호작용결과 시각화

In [ ]:
states = [np.array([0,0])] + agent.next_states[-agent.playtimes[-1]:]
show(states)

# 8. `AgentExplorer`



## A. 클래스 설계

In [ ]:
class AgentExplorer(AgentGreedy):
    def __init__(self,env):
        super().__init__(env)
        self.eps = 0
    def act(self):
        if np.random.rand() < self.eps:
            self.action = self.action_space.sample()
        else:
            super().act()

## B. 환경과 상호작용

In [ ]:
env = GridWorld()
agent = AgentExplorer(env)
agent.eps = 1
for _ in range(5000):
    # Step1: 에피소드 준비
    agent.current_state = env.reset()
    agent.terminated = False
    agent.score = 0
    # Step2: 에프소드 진행
    for t in range(50):
        # step1: 행동
        agent.act()
        # step2: 보상
        agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
        # step3: 저장 & 학습
        agent.save_experience()
        agent.learn()
        # step4: 다음 스텝준비
        agent.current_state = agent.next_state
        if agent.terminated: break
    # Step3: 다음에피소드 준비
    agent.scores.append(agent.score)
    agent.playtimes.append(t+1)
    agent.n_episodes = agent.n_episodes + 1
    agent.eps = agent.eps * 0.999
    #--#
    if (agent.n_episodes % 500) ==0:
        print(
            f"Epsiode: {agent.n_episodes} \t"
            f"Score: {np.mean(agent.scores[-100:])} \t"
            f"Playtime: {np.mean(agent.playtimes[-100:])}\t"
            f"Epsilon: {agent.eps : .2f}"
        )

Epsiode: 500 	Score: -10.3 	Playtime: 3.5	Epsilon:  0.61
Epsiode: 1000 	Score: 2.84 	Playtime: 10.16	Epsilon:  0.37
Epsiode: 1500 	Score: 36.58 	Playtime: 12.81	Epsilon:  0.22
Epsiode: 2000 	Score: 30.7 	Playtime: 15.48	Epsilon:  0.14
Epsiode: 2500 	Score: 29.18 	Playtime: 26.05	Epsilon:  0.08
Epsiode: 3000 	Score: 5.9 	Playtime: 32.43	Epsilon:  0.05
Epsiode: 3500 	Score: 0.01 	Playtime: 36.28	Epsilon:  0.03
Epsiode: 4000 	Score: -26.58 	Playtime: 43.95	Epsilon:  0.02
Epsiode: 4500 	Score: -28.24 	Playtime: 43.86	Epsilon:  0.01
Epsiode: 5000 	Score: -41.0 	Playtime: 48.99	Epsilon:  0.01


## C. 상호작용 결과 시각화

In [ ]:
states = [np.array([0,0])] + agent.next_states[-agent.playtimes[-1]:]
show(states)